# ドレミファソラシドを鳴らす
音を鳴らすためには次のパッケージをインストールする  
PyAudio https://people.csail.mit.edu/hubert/pyaudio/ 

備考：国によって「音名」と「階名」の呼び方は異なる。  
日本人が良く知っているのは音名であり，これはイタリア語が由来である。<br>
イタリア語の呼び方は，ドレミファソラシド（Do, Re, Mi, Fa, Sol, La, Si）<br>
ドイツ語では　C（ツェー），D（デー），E（エー），F（エフ），G（ゲー），A（アー），H（ハー）<br>
英語はドイツ語読みに準じている<br>
Wikipedia https://en.wikipedia.org/wiki/Key_signature_names_and_translations <br>
YAMAHA https://jp.yamaha.com/services/music_pal/study/score/index.html

In [ ]:
# -*- coding: utf-8 -*-
import wave
import struct
import numpy as np
import pyaudio

In [ ]:
def createSinWave (A, f0, fs, length):
    """振幅A,基本周波数f0, サンプリング周波数fs,長さlength秒の正弦波を作成して返す"""
    
    data = []
    for n in np.arange(length * fs):
        s = A * np.sin(2 * np.pi * f0 * n / fs) # 周波数f0 の正弦波
        if s > 1.0: s = 1.0    #正弦波の振幅-1～＋１に制限
        if s < -1.0: s = -1.0
        data.append(s)

    data = [int(x * 32767.0) for x in data] # [-32768, 32767]の整数に変換
    # list に * をつけて引数展開
    data = struct.pack("h" * len(data), *data) # バイナリデータに変換
    return data

def play(data, fs, bit):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
               channels=1,
               rate=int(fs),
               output=True)

    chunk = 1024
    sp = 0
    buffer = data[sp:sp+chunk]
    while buffer != b'':
        stream.write(buffer)
        sp = sp + chunk
        buffer = data[sp:sp+chunk]

    stream.close()
    p.terminate()


if __name__ == "__main__" :
    freqList = [262, 294, 330, 349, 392, 440, 494, 523]
#               ド,  レ,  ミ, ファ  ソ,  ラ,  シ,  ド　
    for f in freqList:
        data = createSinWave(1.0, f, 8000.0, 1.0)
        play(data, 8000, 16)